## Tidy Crop

In [29]:
import pandas as pd

land_data = pd.read_excel("附件1.xlsx").drop(columns=['说明 '])
crop_data = pd.read_excel("附件1.xlsx", sheet_name="crop_tidied")
# display(land_data)
# display(crop_data)

In [39]:
import pandas as pd

# Function to split land and season
def split_land_season(land_season_str):
    pairs = land_season_str.split('|')
    result = []
    for pair in pairs:
        if '-' in pair:
            land, *seasons = pair.split('-')
            for season in seasons:
                result.append(f'{land}-{season}')
        else:
            result.append(f"{pair}-单季")
    return result

# Apply the function to each row and explode the list into separate rows
crop_data['land_season_split'] = crop_data['种植耕地'].apply(split_land_season)
crop_data_exploded = crop_data.explode('land_season_split')

# Create dummy variables
dummies = pd.get_dummies(crop_data_exploded['land_season_split'])

# Group back to original rows by summing the dummy variables
crop_data_dummies = dummies.groupby(crop_data_exploded.index).sum()

# Concatenate the original DataFrame with the dummy variables
crop_data_final = pd.concat([crop_data, crop_data_dummies], axis=1)

# Drop the intermediate column
crop_data_final = crop_data_final.drop(columns=['land_season_split'])

# Display the final DataFrame
crop_data_final.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 41 entries, 0 to 40
Data columns (total 14 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   作物编号      41 non-null     int64 
 1   作物名称      41 non-null     object
 2   作物类型      41 non-null     object
 3   种植耕地      41 non-null     object
 4   山坡地-单季    41 non-null     int64 
 5   平旱地-单季    41 non-null     int64 
 6   普通大棚-第一季  41 non-null     int64 
 7   普通大棚-第二季  41 non-null     int64 
 8   智慧大棚-第一季  41 non-null     int64 
 9   智慧大棚-第二季  41 non-null     int64 
 10  梯田-单季     41 non-null     int64 
 11  水浇地-单季    41 non-null     int64 
 12  水浇地-第一季   41 non-null     int64 
 13  水浇地-第二季   41 non-null     int64 
dtypes: int64(11), object(3)
memory usage: 4.6+ KB


In [35]:
import lib.excel_util as excel_util

workbook_path = r'\\wsl.localhost\Ubuntu-22.04\home\flas\source\course\modelling\agristrat\附件1.xlsx'
sheet_name = 'land'

excel_util.export_to_excel(land_data, workbook_path, sheet_name)


## Generate Configs & Price

In [44]:
import pandas as pd

# Sample data (replace this with your actual DataFrame)
crop_data_final

# List of耕地类型 and季次 columns
land_season_columns = [
    '山坡地-单季', '平旱地-单季', '普通大棚-第一季', '普通大棚-第二季',
    '智慧大棚-第一季', '智慧大棚-第二季', '梯田-单季', '水浇地-单季',
    '水浇地-第一季', '水浇地-第二季'
]

# Create an empty list to store the results
results = []

# Iterate over each row in the DataFrame
for index, row in crop_data_final.iterrows():
    crop_name = row['作物名称']
    
    # Iterate over each耕地类型 and季次 column
    for col in land_season_columns:
        if row[col] == 1:  # Check if the dummy variable is 1
            # Split the column name to get耕地类型 and季次
            land_type, season = col.split('-')
            
            # Append the result as a tuple (作物名称, 耕地类型, 季次)
            results.append((crop_name, land_type, season))

# Create a new DataFrame from the results
configs = pd.DataFrame(results, columns=['作物名称', '耕地类型', '季次'])
configs.sort_values("耕地类型", inplace=True)

# Display the final DataFrame
configs.info()

<class 'pandas.core.frame.DataFrame'>
Index: 125 entries, 0 to 61
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   作物名称    125 non-null    object
 1   耕地类型    125 non-null    object
 2   季次      125 non-null    object
dtypes: object(3)
memory usage: 3.9+ KB


In [45]:
excel_util.export_to_excel(configs, workbook_path, "configs")